In [237]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={"lines.linewidth": 2}, palette  = "deep", style = "ticks")
from itertools import product, permutations, combinations, combinations_with_replacement
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt

In [266]:
model1 = "filter"  #non_filter OR filter OR multiply OR score
model2 = "multiply"  #non_filter OR filter OR multiply OR score
hvg_version = "sct"  #sct OR normdata
data_version = "count"  #norm OR count
directory = "/home/zc354/network_2000/"

pred_path_1=directory+model1+'_'+hvg_version+'_'+data_version+'.csv'
pred_path_2=directory+model2+'_'+hvg_version+'_'+data_version+'.csv'
true_path='/gpfs/gibbs/pi/zhao/yw599/GRN/BEELINE-Networks/Networks/human/Non-specific-ChIP-seq-network.csv'
directed=True
#save_path='/gpfs/gibbs/pi/zhao/zc354/GRN/output'
#save_name='/AUPRC_TF_Keran_indirected'

In [267]:
#Read true netword and predicted network
trueEdgesDF = pd.read_csv(true_path,sep = ',', header = 0, index_col = None)
predEdgesDF_1 = pd.read_csv(pred_path_1, sep = ',', header =  0, index_col = None)
predEdgesDF_2 = pd.read_csv(pred_path_2, sep = ',', header =  0, index_col = None)

In [268]:
len(predEdgesDF_1)

169494

In [269]:
len(predEdgesDF_2)

169494

In [270]:
#Unification changes lowercase letters to uppercase letters
trueEdgesDF['Gene1'] = trueEdgesDF['Gene1'].str.upper()
trueEdgesDF['Gene2'] = trueEdgesDF['Gene2'].str.upper()

predEdgesDF_1['Gene1'] = predEdgesDF_1['Gene1'].str.upper()
predEdgesDF_1['Gene2'] = predEdgesDF_1['Gene2'].str.upper()

predEdgesDF_2['Gene1'] = predEdgesDF_2['Gene1'].str.upper()
predEdgesDF_2['Gene2'] = predEdgesDF_2['Gene2'].str.upper()

In [271]:
#Drop index in predEdgesDF that Edgeweight=0
predEdgesDF_1.drop(predEdgesDF_1[predEdgesDF_1['Edgeweight'] == 0].index,inplace=True)

predEdgesDF_2.drop(predEdgesDF_2[predEdgesDF_2['Edgeweight'] == 0].index,inplace=True)

In [272]:
#Drop selfEdges
trueEdgesDF = trueEdgesDF.loc[(trueEdgesDF['Gene1'] != trueEdgesDF['Gene2'])]
predEdgesDF_1 = predEdgesDF_1.loc[(predEdgesDF_1['Gene1'] != predEdgesDF_1['Gene2'])]

predEdgesDF_2 = predEdgesDF_2.loc[(predEdgesDF_2['Gene1'] != predEdgesDF_2['Gene2'])]

In [273]:
#Drop duplicates in trueEdgesDF and predEdgesDF
trueEdgesDF.drop_duplicates(keep = 'first', inplace=True)
predEdgesDF_1.drop_duplicates(keep = 'first',subset=['Gene1','Gene2'],inplace=True)
predEdgesDF_2.drop_duplicates(keep = 'first',subset=['Gene1','Gene2'],inplace=True)

In [274]:
print(len(set(trueEdgesDF['Gene1'])))
print(len(set(trueEdgesDF['Gene2'])))

print(len(set(predEdgesDF_1['Gene1'])))
print(len(set(predEdgesDF_1['Gene2'])))

print(len(set(predEdgesDF_2['Gene1'])))
print(len(set(predEdgesDF_2['Gene2'])))

2157
23377
387
1762
383
1762


In [275]:
true_TF = set(trueEdgesDF['Gene1'])
true_tg = set(trueEdgesDF['Gene2'])

pred_TF_1 = set(predEdgesDF_1['Gene1'])
pred_tg_1 = set(predEdgesDF_1['Gene2'])

pred_TF_2 = set(predEdgesDF_2['Gene1'])
pred_tg_2 = set(predEdgesDF_2['Gene2'])

In [276]:
TFset_1 = (true_TF & pred_TF_1) 
tgset_1 = (true_tg & pred_tg_1)

TFset_2 = (true_TF & pred_TF_2) 
tgset_2 = (true_tg & pred_tg_2)

In [277]:
print(len(TFset_1))
print(len(TFset_2))
print(len(tgset_1))
print(len(tgset_2))

197
194
1571
1571


In [278]:
TFset = TFset_1 & TFset_2
tgset = tgset_1 & tgset_2

In [279]:
print(len(TFset))
print(len(tgset))

194
1571


In [280]:
trueEdgesDF = trueEdgesDF[(trueEdgesDF['Gene1'].isin(TFset_2))&(trueEdgesDF['Gene2'].isin(tgset_2))]
predEdgesDF_1 = predEdgesDF_1[(predEdgesDF_1['Gene1'].isin(TFset_2))&(predEdgesDF_1['Gene2'].isin(tgset_2))]
predEdgesDF_2 = predEdgesDF_2[(predEdgesDF_2['Gene1'].isin(TFset_2))&(predEdgesDF_2['Gene2'].isin(tgset_2))]

In [281]:
len(predEdgesDF_1)

58665

In [282]:
len(predEdgesDF_2)

58093

In [283]:
#Generate all possibleEdges using Cartesian product of trueEdgesDF['Gene1'] & trueEdgesDF['Gene2']
possibleEdges = set(product(set(trueEdgesDF['Gene1']),set(trueEdgesDF['Gene2'])))
possibleEdgesDF = pd.DataFrame(list(possibleEdges))
possibleEdgesDF.columns=['Gene1','Gene2']

In [284]:
#directed
TrueEdgesDF = pd.DataFrame(columns=['Edge'])
TrueEdgesDF['Edge'] = trueEdgesDF['Gene1'].map(str) +"|"+ trueEdgesDF['Gene2'].map(str)
PredEdgesDF_1 = pd.DataFrame(columns=['Edge','Weight'])
PredEdgesDF_1['Edge'] = predEdgesDF_1['Gene1'].map(str) +"|"+ predEdgesDF_1['Gene2'].map(str)
PredEdgesDF_1['Weight'] = predEdgesDF_1['Edgeweight']

PredEdgesDF_2 = pd.DataFrame(columns=['Edge','Weight'])
PredEdgesDF_2['Edge'] = predEdgesDF_2['Gene1'].map(str) +"|"+ predEdgesDF_2['Gene2'].map(str)
PredEdgesDF_2['Weight'] = predEdgesDF_2['Edgeweight']

In [285]:
PossibleEdgesDF = pd.DataFrame(columns=['Edge','Pred','True'])
PossibleEdgesDF['Edge'] = possibleEdgesDF['Gene1'].map(str) +"|"+ possibleEdgesDF['Gene2'].map(str)

In [286]:
#Generate the set of true/pred/possible edges
TrueEdgesset=set(TrueEdgesDF['Edge'])

PredEdgesset_1=set(PredEdgesDF_1['Edge'])
PredEdgesset_2=set(PredEdgesDF_2['Edge'])

PossibleEdgesset=set(PossibleEdgesDF['Edge'])

PredEdgesDF_1.set_index('Edge',inplace=True)
PredEdgesDF_2.set_index('Edge',inplace=True)
TrueEdgesDF.set_index('Edge',inplace=True)
PossibleEdgesDF.set_index('Edge',inplace=True)

In [49]:
def gen_auprc(TrueEdgesset = TrueEdgesset, PredEdgesset = PredEdgesset_1, PossibleEdgesset = PossibleEdgesset,PredEdgesDF=PredEdgesDF_1,TrueEdgesDF =TrueEdgesDF,PossibleEdgesDF=PossibleEdgesDF ):
    inbothset = (TrueEdgesset & PredEdgesset) & PossibleEdgesset
    onlyintrue = (TrueEdgesset - PredEdgesset) & PossibleEdgesset
    onlyinpred = (PredEdgesset - TrueEdgesset) & PossibleEdgesset
    inneitherset = PossibleEdgesset - (TrueEdgesset | PredEdgesset)
    
    #Assign values to different types of edges
    inbothDF = PredEdgesDF.loc[list(inbothset),:]
    inbothDF['True'] = 1
    inbothDF.columns = ['Pred','True']
    
    onlyintrueDF = TrueEdgesDF.loc[list(onlyintrue),:]
    onlyintrueDF['Pred'] = 0
    onlyintrueDF['True'] = 1
    
    onlyinpredDF = PredEdgesDF.loc[list(onlyinpred),:]
    onlyinpredDF['True'] = 0
    onlyinpredDF.columns = ['Pred','True']
    
    inneitherDF = PossibleEdgesDF.loc[list(inneitherset),:]
    inneitherDF['Pred'] = 0
    inneitherDF['True'] = 0
    
    print('lenth of inbothDF:'+str(len(inbothDF)))
    print('lenth of onlyintrueDF:'+str(len(onlyintrueDF)))
    print('lenth of onlyinpredDF:'+str(len(onlyinpredDF)))
    print('lenth of trueDF:'+str(len(onlyintrueDF)+len(inbothDF)))
    
    outDF = pd.concat([inbothDF,onlyintrueDF,onlyinpredDF,winneitherDF],axis=0)
    print('length of outDF:'+str(len(outDF)))
    
    prec, recall, thresholds = precision_recall_curve(y_true=outDF['True'],probas_pred=outDF['Pred'], pos_label=1)
    
    return auc(recall,prec)

In [50]:
gen_auprc(PredEdgesset = PredEdgesset_1, PredEdgesDF=PredEdgesDF_1)

lenth of inbothDF:5287
lenth of onlyintrueDF:22
lenth of onlyinpredDF:164013
lenth of trueDF:5309
length of outDF:172032


0.03908607232001104

In [51]:
gen_auprc(PredEdgesset = PredEdgesset_2, PredEdgesDF=PredEdgesDF_2)

lenth of inbothDF:1427
lenth of onlyintrueDF:3882
lenth of onlyinpredDF:31020
lenth of trueDF:5309
length of outDF:172032


0.041577088965947334

In [89]:
#Generate four possible conditions using set operation
#inbothset represents edges in both true and pred network
#onlyintrue set represents edges in true network but not in pred
#onlyinpred set represents edges in pred but not in true network
#inneitherset represent edges in neither of true and pred network but in possible network
inbothset = (TrueEdgesset & PredEdgesset) & PossibleEdgesset
onlyintrue = (TrueEdgesset - PredEdgesset) & PossibleEdgesset
onlyinpred = (PredEdgesset - TrueEdgesset) & PossibleEdgesset
inneitherset = PossibleEdgesset - (TrueEdgesset | PredEdgesset)

In [90]:
#Set Edges as index in three DataFrames to facilitate indexing
PredEdgesDF_1.set_index('Edge',inplace=True)
TrueEdgesDF_1.set_index('Edge',inplace=True)
PossibleEdgesDF_1.set_index('Edge',inplace=True)

In [91]:
#Assign values to different types of edges
inbothDF = PredEdgesDF_1.loc[list(inbothset),:]
inbothDF['True'] = 1
inbothDF.columns = ['Pred','True']

In [92]:
onlyintrueDF = TrueEdgesDF_1.loc[list(onlyintrue),:]
onlyintrueDF['Pred'] = 0
onlyintrueDF['True'] = 1

In [93]:
onlyinpredDF = PredEdgesDF_1.loc[list(onlyinpred),:]
onlyinpredDF['True'] = 0
onlyinpredDF.columns = ['Pred','True']

In [94]:
inneitherDF = PossibleEdgesDF_1.loc[list(inneitherset),:]
inneitherDF['Pred'] = 0
inneitherDF['True'] = 0

In [95]:
outDF = pd.concat([inbothDF,onlyintrueDF,onlyinpredDF,inneitherDF],axis=0)

In [96]:
prec, recall, thresholds = precision_recall_curve(y_true=outDF['True'],probas_pred=outDF['Pred'], pos_label=1)

In [97]:
auc(recall, prec)

0.043745304596455876

In [437]:
auprc=auc(recall, prec)

In [19]:
#indircted
#Generate outDF copied from possibleEdgesDF
outDF = pd.DataFrame(columns=['Edge','Pred','True'])
outDF['Edge'] = possibleEdgesDF['Gene1'].map(str) +"|"+ possibleEdgesDF['Gene2'].map(str)

In [20]:
def gen_dict():
    gene2_dict={}
    for gene2,df in trueEdgesDF.groupby('Gene2'):
        gene2_dict[gene2]=set(df['Gene1'])
    return gene2_dict
gene2_dict=gen_dict()

In [21]:
#Generate a function of searching for trueEdgesDF
def main_fun(row):
    global gene2_dict
    t1,t2=row['Gene1'],row['Gene2']
    # First search for (gene1==t1 & gene2==t2)
    if gene2_dict.get(t2):
        temp=gene2_dict[t2]
        if t1 in temp:
            return 1
          # Then search for (gene1==t2 & gene2==t1)
    if gene2_dict.get(t1):
        temp=gene2_dict[t1]
        if t2 in temp:
             return 1
    return 0

In [22]:
import modin.pandas 
import swifter
swifter.register_modin()

In [23]:
#Using apply function to speed up assigning values
Value=possibleEdgesDF.swifter.apply(main_fun,axis=1)

Pandas Apply:   0%|          | 0/109224 [00:00<?, ?it/s]

In [24]:
#Assigning values for TrueEdges
outDF['True'] = Value

In [25]:
def gen_dict_2():
    gene2_dict_2={}
    for gene2,df in predEdgesDF.groupby('Gene2'):
        gene2_dict_2[gene2]=dict(zip(list(df['Gene1']),list(df['Edgeweight'])))
    return gene2_dict_2
gene2_dict_2 = gen_dict_2()

In [26]:
def main_fun_2(row):
    global gene2_dict_2
    #res to record EdgeWeights in predEdgesDF
    res=[]
    t1 ,t2=row['Gene1'],row['Gene2']
    # First search for (gene1==t1 & gene2==t2)
    if gene2_dict_2.get(t2):
        temp=gene2_dict_2[t2]
        if t1 in temp:
            res.append(temp[t1])
            
    # Then search for (gene1==t2 & gene2==t1)
    if gene2_dict_2.get(t1):
        temp=gene2_dict_2[t1]
        if t2 in temp:
            res.append(temp[t2])
    if res:
        return max(res)
    else:
        return 0

In [27]:
swifter.register_modin()
Weight = possibleEdgesDF.swifter.apply(main_fun_2,axis=1)

Pandas Apply:   0%|          | 0/109224 [00:00<?, ?it/s]

In [28]:
outDF['Pred'] = Weight

In [29]:
outDF.set_index('Edge',inplace = True)

In [30]:
prec, recall, thresholds = precision_recall_curve(y_true=outDF['True'],probas_pred=outDF['Pred'], pos_label=1)

In [31]:
auprc = auc(recall, prec)

In [116]:
plt.xlim(0,1)    
plt.ylim(0,1)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.plot(prec, recall, "blue")  
plt.savefig(save_path+save_name+'.pdf')
plt.savefig(save_path+save_name+'.png')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [32]:
print(save_name+':'+str(auprc))

/AUPRC_TF_Keran_indirected:0.04654243225256931
